# Chapter 9: Learning and Adaptation

Key Takeaways:
- **Evolutionary Coding** uses LLMs to automatically discover and optimize algorithms through iterative code generation, evaluation, and selection.
- **OpenEvolve** is an open-source implementation of evolutionary coding that can evolve entire code files across multiple programming languages.
- **Multi-objective Optimization** allows simultaneous optimization of multiple metrics like correctness, performance, and code quality.

### Heuristic: *Evolution over manual optimization.*

## Setup and Initialization

In [ ]:
import os
import nest_asyncio
from dotenv import load_dotenv

# Allow nested event loops (required for OpenEvolve in Jupyter)
nest_asyncio.apply()

load_dotenv()

# --- Configuration ---
PROJECT_ROOT = os.path.dirname(os.getcwd())
SCRIPTS_DIR = os.path.join(PROJECT_ROOT, "scripts")

print(f"✅ Configuration Loaded:")
print(f"   Project Root: {PROJECT_ROOT}")
print(f"   Scripts Directory: {SCRIPTS_DIR}")

## OpenEvolve Overview

OpenEvolve is an evolutionary coding agent that leverages Large Language Models (LLMs) to automatically optimize and discover algorithms. It implements the key concepts from Google DeepMind's AlphaEvolve system.

### Key Components

| Component | Description |
|-----------|-------------|
| **LLM Ensemble** | Uses multiple language models to generate diverse code modifications |
| **Prompt Sampler** | Creates context-rich prompts incorporating past programs and scores |
| **Evaluator Pool** | Tests generated programs and assigns scores based on defined metrics |
| **Program Database** | Stores evolved programs using MAP-Elites algorithm for diversity |

## The Evolution Pipeline

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Initial Code   │────▶│   LLM Mutates   │────▶│    Evaluate     │
│   (Program)     │     │   (Generate)    │     │   (Score)       │
└─────────────────┘     └─────────────────┘     └────────┬────────┘
                                                         │
                                                         ▼
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Best Program   │◀────│    Selection    │◀────│ Program Database│
│   (Output)      │     │   (Fitness)     │     │   (Archive)     │
└─────────────────┘     └─────────────────┘     └─────────────────┘
```

## Required Files

OpenEvolve requires three key files to operate:

1. **Initial Program** (`initial_program.py`) - The starting code to evolve
2. **Evaluator** (`evaluator.py`) - Defines how to score evolved programs
3. **Config** (`config.yaml`) - Evolution parameters and LLM settings

### 1. Initial Program

This is the starting point for evolution. Mark sections with `EVOLVE-BLOCK-START` and `EVOLVE-BLOCK-END` comments to indicate which parts can be modified.

In [ ]:
# View the initial program
initial_program_path = os.path.join(SCRIPTS_DIR, "initial_program.py")

with open(initial_program_path, 'r') as f:
    print(f.read())

### 2. Evaluator

The evaluator defines the fitness function that scores each evolved program. It should return a dictionary of metrics.

In [ ]:
# View the evaluator
evaluator_path = os.path.join(SCRIPTS_DIR, "evaluator.py")

with open(evaluator_path, 'r') as f:
    print(f.read())

### 3. Configuration

The config file controls evolution parameters, LLM settings, and evaluation options.

In [ ]:
# View the configuration
config_path = os.path.join(SCRIPTS_DIR, "config.yaml")

with open(config_path, 'r') as f:
    print(f.read())

## Running Evolution

OpenEvolve provides a simple `run_evolution` API that accepts file paths directly:

In [ ]:
from openevolve import run_evolution

# Define file paths
initial_program_path = os.path.join(SCRIPTS_DIR, "initial_program.py")
evaluator_path = os.path.join(SCRIPTS_DIR, "evaluator.py")
config_path = os.path.join(SCRIPTS_DIR, "config.yaml")

print("📁 OpenEvolve Files:")
print(f"   Initial Program: {initial_program_path}")
print(f"   Evaluator: {evaluator_path}")
print(f"   Config: {config_path}")
print("\n✅ Ready to run evolution!")

In [ ]:
# Run the evolution process
# Note: This requires an LLM API key (e.g., OPENAI_API_KEY or GOOGLE_API_KEY)
# In a real scenario, you would run many more iterations

result = run_evolution(
    initial_program=initial_program_path,
    evaluator=evaluator_path,
    config=config_path,
    iterations=2,  # Small number for demo; use 100+ in production
    output_dir="../scripts/openevolve_output"
)

print(f"\n🏆 Evolution Complete!")
print(f"Best program metrics:")
for name, value in result.metrics.items():
    print(f"  {name}: {value:.4f}")

## Analyzing Results

After evolution, you can examine the best program and its improvements:

In [ ]:
# View the evolved code
print("📝 Evolved Program Code:")
print("=" * 50)
print(result.best_code)
print("=" * 50)

# Compare metrics with the original
print(f"\n📊 Performance Improvement:")
print(f"  Correctness: {result.metrics.get('correctness', 0):.2%}")
print(f"  Performance: {result.metrics.get('performance', 0):.2%}")
print(f"  Overall Fitness: {result.metrics.get('fitness', 0):.2%}")

## Alternative: Using the Low-Level API

For more control, you can use the `OpenEvolve` class directly with a `Config` object:

In [ ]:
from openevolve import OpenEvolve
from openevolve.config import Config

# Load configuration from YAML file
config = Config.from_yaml(os.path.join(SCRIPTS_DIR, "config.yaml"))

# Initialize the OpenEvolve controller
evolve = OpenEvolve(
    initial_program_path=os.path.join(SCRIPTS_DIR, "initial_program.py"),
    evaluation_file=os.path.join(SCRIPTS_DIR, "evaluator.py"),
    config=config
)

print("✅ OpenEvolve initialized with Config object!")

## Key Configuration Options

| Parameter | Description | Typical Range |
|-----------|-------------|---------------|
| `iterations` | Number of evolution cycles | 100-10000 |
| `population_size` | Programs maintained per generation | 10-100 |
| `temperature` | LLM creativity (higher = more diverse) | 0.5-1.0 |
| `mutation_rate` | Probability of code modification | 0.1-0.5 |
| `elite_count` | Top programs preserved each generation | 1-5 |

## Best Practices

1. **Start Simple** - Begin with a basic, working implementation
2. **Clear Metrics** - Define measurable, unambiguous evaluation criteria
3. **Balanced Objectives** - Weight correctness higher than performance initially
4. **Checkpointing** - Save progress regularly for long evolution runs
5. **Diversity** - Use the MAP-Elites algorithm to avoid local optima

## Conclusion

Learning and Adaptation through evolutionary coding represents a paradigm shift in algorithm development. Instead of manually optimizing code, you can:

- **Define the goal** (evaluation metrics)
- **Provide a starting point** (initial program)
- **Let evolution discover improvements** (run OpenEvolve)

This approach is particularly powerful for complex optimization problems where human intuition may miss novel solutions.